In [1]:
import requests
import pandas as pd
import json
import re
import polars as pl
import Enum_data as ed
pl.Config.set_tbl_rows(1000)
from persiantools import characters, digits
import sqlite3

import general_functions as gf

### Phase 2

#### خواندن جدول در لینک ارسال شده

In [4]:
def get_table(url: str, table: int):
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:143.0) Gecko/20100101 Firefox/143.0',
    'Accept': 'application/json, text/plain, */*',
    'Accept-Language': 'en-US,en;q=0.5',
    'Accept-Encoding': 'gzip, deflate, br, zstd',
    'Origin': 'https://www.codal.ir',
    'Connection': 'keep-alive',
    'Referer': 'https://www.codal.ir/',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-site',
    'Cookie': 'TS018fb0f7=01f9930bd2e2675d04882f623c888052df60031e7775d8e3c459dc4cb96bf8e870e0a9bf0e0eedff68bcce23fd9701c7a7fc0855c4; Unknown=1076170924.20480.0000'
    }
    response = requests.request("GET", url)
    statement = response.text
    pattern = r"var datasource = (.*?});"
    match = re.search(pattern, statement)
    if match:
        text = match.group(1)
    records = []
    records.append(
        (statement, text))
    for _, data in records:
        continue
    items = json.loads(data)['sheets']

    if isinstance(table, list):
        cells = []
        for t in table:
            raw_cells = items[0]['tables'][t]['cells']
            cells.append([(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in raw_cells])
        return [x for xs in cells for x in xs]
    
    cells = items[0]['tables'][table]['cells']
    return [(i['columnSequence'], i['rowSequence'], i['value'], i['periodEndToDate']) for i in cells]

def create_dict_dataframes(url: str, date: int, report_type: str) -> dict:
    all_data ={'report_last_year': pl.DataFrame(),
            'report_this_year': pl.DataFrame(),
            'est_remain': pl.DataFrame(),
            'est_next_year': pl.DataFrame()}

    cells_tuples = get_table(url, ed.tabels[report_type].value)
    dates = sorted(list(set([i[-1] for i in cells_tuples if i[-1] != ''])))
           
    for date_ in dates:
        if report_type=='RawMaterial':
            products = []
            filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == date_]
            products = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '']
            df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
            df = df.pivot(values="value", on="col", index="row").sort("row")
            
            df_products = pl.from_records(products, schema=["col", "row", "value"], orient="row")
            df_products = df_products.pivot(values="value", on="col", index="row").sort("row")

            df_products = df_products.filter(df_products['row']>=df['row'][0] ,df_products['row']<=df['row'][-1])
            df = df_products.join(df, on="row", how="left")

            display(df)
            
        else:
            filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '' or i[-1] == (date_)]
            df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
            df = df.pivot(values="value", on="col", index="row").sort("row")
            df = df.with_columns(pl.col('1').map_elements(characters.ar_to_fa, return_dtype=pl.String))

        if report_type == 'Operational':
            if(df.width < 8):
                continue
        
        for i, col in enumerate(df.columns):
            cols = ed.cols[report_type].value
            value = df[col][0]        # value from first row
            if(isinstance(value, str)):
                for val in value.split():
                    if'/' in val:
                        num = int(val.replace('/', ''))
                        next_cols = df.columns[i : i + cols + 1]
                        cols_to_select = ['1','2', *next_cols] if report_type in ['Operational'] else ['1', *next_cols]
                        if num < date :
                            all_data['report_last_year'] = df.select(cols_to_select)
                            all_data['report_last_year'] = all_data['report_last_year'].with_row_index("row")
                        elif num == date :
                            all_data['report_this_year'] = df.select(cols_to_select)
                            all_data['report_this_year'] = all_data['report_this_year'].with_row_index("row")
                        else:
                            if(val[:4] == str(date)[:4]):
                                all_data['est_remain'] = df.select(cols_to_select)
                                all_data['est_remain'] = all_data['est_remain'].with_row_index("row")
                            else:
                                all_data['est_next_year'] = df.select(cols_to_select)
                                all_data['est_next_year'] = all_data['est_next_year'].with_row_index("row")
    return all_data

def create_operational_dataframe(symbol: str, url: str, date: int, period: int, publish: int):
    all_data = create_dict_dataframes(url, date, 'Operational')
    for name, df in all_data.items():
        df = df.rename({c: str(i) for i, c in enumerate(df.columns)})
        df = df.rename({'0': 'row'})
        table_date = [d for d in str(df[df.columns[3]][0]).split() if '/' in d][0]
        df_internal = df.clone()
        idx_internal = df_internal.filter(df_internal['1'] == "فروش داخلی:")['row'][0]
        idx_total_internal = df_internal.filter(df_internal['1'] == "جمع فروش داخلی")['row'][0]
        df_internal = df_internal.filter(df_internal['row'] > idx_internal, df_internal['row'] < idx_total_internal-1)[['1','2']]
        df_internal=df_internal.with_columns(pl.lit("داخلی").alias("3"))
        df_export=df.clone()
        idx_export = df_export.filter(df_export['1'] == "فروش صادراتی:")['row'][0]
        idx_total_export = df_export.filter(df_export['1'] == "جمع فروش صادراتی")['row'][0]
        df_export = df_export.filter(df_export['row']>idx_export ,df_export['row']<idx_total_export -1)[['1','2']]
        df_export = df_export.with_columns(pl.lit("صادراتی").alias('3'))
        df_products = pl.concat([df_internal,df_export])
        for i in df[0].rows(named=True):
            for k, v in i.items():
                if '/' in str(v):
                    col_num_production = k
                    dummy_data = df[col_num_production:str(int(col_num_production)+2)]
                    dummy_data = dummy_data.insert_column(0, df['row'])
                            
                    sales_internal = dummy_data.filter(dummy_data['row'] > idx_internal, dummy_data['row'] < idx_total_internal-1)
                    sales_export = dummy_data.filter(dummy_data['row']>idx_export ,dummy_data['row']<idx_total_export -1)
                    sales_products = pl.concat([sales_internal, sales_export])               
                    

                    #Rename columns to range(0,1,...)
                    sales_products = sales_products.rename(dict(zip(sales_products.columns, map(str, range(len(sales_products.columns))))))
                    #If "" is in the dataframe
                    display(sales_products)
                    if gf.has_empty_string(sales_products):
                        continue
                    #Convert string to Int from Col 1 to end
                    sales_products = sales_products.with_columns([pl.col(sales_products.columns).cast(pl.Int32)])
                    # concat label and values
                    df_products = df_products.rename({c: "_" + c for c in df_products.columns})
                    
                    data = pl.concat([df_products, sales_products], how="horizontal")
                
                    data.columns = [str(i) for i in range(data.width)]
                    data = data.insert_column(0, pl.lit(symbol).alias("Symbol"))
                    data = data.insert_column(1, pl.lit(int(digits.fa_to_en(table_date).replace('/', ''))).alias("Date"))
                    data = data.insert_column(2, pl.lit(period).alias("Period"))
                    data = data.insert_column(3, pl.lit(publish).alias("Publish"))
                    data = data.rename({
                        "0": "Product",
                        "1": "Unit",
                        "2": "Type",
                        "3":"Production",
                        "4":"Sales",
                        "5":"Price"
                    })
                    all_data[name] = data
    return all_data

def create_cogs_dataframe(symbol: str, url: str, date: int, period: int, publish: int) -> dict:
    all_data = create_dict_dataframes(url, date, 'COGS')

    col_names =["description",
                "Direct Materials Used",
                "Direct Labor",
                "Manufacturing Overhead",
                "Total",
                "Under-absorbed Manufacturing Overhead",
                "Total Manufacturing Costs",
                "Net Work-in-Process Inventory",
                "Abnormal Waste",
                "Cost of Goods Manufactured",
                "Beginning Finished Goods Inventory",
                "Ending Finished Goods Inventory",
                "Cost of Goods Sold (COGS)",
                "Cost of Services Rendered",
                "Total Cost"
                ]
    for key, data in all_data.items():
        data = data.transpose(include_header=False)
        data = data.rename(dict(zip(data.columns, col_names)))
        data = data.tail(1)
        
        table_date = [d for d in str(data.row(0)[0]).split() if '/' in d][0]
        data = data.drop(data.columns[0])
        data = data.insert_column(0, pl.lit(symbol).alias("Symbol"))
        data = data.insert_column(1, pl.lit(int(digits.fa_to_en(table_date).replace('/', ''))).alias("Date"))
        data = data.insert_column(2, pl.lit(period).alias("Period"))
        data = data.insert_column(3, pl.lit(publish).alias("Publish"))
        all_data[key] = data
    return all_data 
  
def convert_df_db_format(df: pl.DataFrame, report_type: str) -> pl.DataFrame:
    print(report_type)
    conn=sqlite3.connect("codal.sqlite")
    names = ['Symbol', 'Product', 'Type', 'Unit']
    if report_type in ['Operational']:
        col_names = ['key','Symbol_id','Date','Period', 'Publish','Product_id','Type_id','Unit_id','Production','Sales','Price']
        for name in names:
            rows = conn.execute(f"SELECT name, id FROM {name}").fetchall()
            lookup = {key: value for key, value in rows}
            df = df.with_columns(pl.col(f"{name}").replace(lookup).alias(f"{name}_id"))
        key_df = df.with_columns((pl.col("Date").cast(pl.Utf8)+ pl.col("Symbol_id").cast(pl.Utf8)+pl.col("Product_id").cast(pl.Utf8)+pl.col("Type_id").cast(pl.Utf8)).alias("key").cast(pl.Int64))
    else:
        col_names = ['key','Symbol_id','Date','Period', 'Publish',
                "Direct Materials Used",
                "Direct Labor",
                "Manufacturing Overhead",
                "Total",
                "Under-absorbed Manufacturing Overhead",
                "Total Manufacturing Costs",
                "Net Work-in-Process Inventory",
                "Abnormal Waste",
                "Cost of Goods Manufactured",
                "Beginning Finished Goods Inventory",
                "Ending Finished Goods Inventory",
                "Cost of Goods Sold (COGS)",
                "Cost of Services Rendered",
                "Total Cost"
                ]
        rows = conn.execute(f"SELECT name, id FROM Symbol").fetchall()
        lookup = {key: value for key, value in rows}
        df = df.with_columns(pl.col(f"Symbol").replace(lookup).alias(f"Symbol_id"))
        key_df = df.with_columns((pl.col("Date").cast(pl.Utf8)+ pl.col("Symbol_id").cast(pl.Utf8)).alias("key").cast(pl.Int64))
    key_df = key_df[col_names]
    key_df = key_df.with_columns(pl.all().cast(pl.Int64))
    return key_df
    

Test Run

In [2]:
#---------------------------------------
symbol = "فخوز"
financial_year = 14031230
report_type = ed.names.COGS.value
#----------------------------------------

sheet_num = ed.sheets[report_type].value
table = ed.tabels[report_type].value
reports = gf.get_results(symbol=symbol, parse_date=gf.parse_date_persian(financial_year), sheet_num=sheet_num)
#reports
date = 14030930
period = int(reports[date]['period'])
publish = int(reports[date]['publish'])
url = reports[date]['url']

NameError: name 'ed' is not defined

In [5]:
all_data = create_dict_dataframes(url, date, report_type)

In [ ]:
all_data['report_this_year']

In [114]:
#---------------------------------------
symbol = "فخوز"
financial_year = 14031230
report_type = ed.names.RawMaterial.value
#----------------------------------------

sheet_num = ed.sheets[report_type].value
table = ed.tabels[report_type].value
reports = get_results(symbol=symbol, parse_date=parse_date_persian(financial_year), sheet_num=sheet_num)
#reports
date = 14030930
period = int(reports[date]['period'])
publish = int(reports[date]['publish'])
url = reports[date]['url']

In [ ]:
cells_tuples = create_dict_dataframes(url, date, 'RawMaterial')


In [ ]:
all_data ={'report_last_year': pl.DataFrame(),
            'report_this_year': pl.DataFrame(),
            'est_remain': pl.DataFrame(),
            'est_next_year': pl.DataFrame()}

dates = sorted(list(set([i[-1] for i in cells_tuples if i[-1] != ''])))

products = []
for date_ in dates:
    filtered_cells = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == date_]
    products = [(i[0], i[1], i[2]) for i in cells_tuples if i[-1] == '']
    df = pl.from_records(filtered_cells, schema=["col", "row", "value"], orient="row")
    df = df.pivot(values="value", on="col", index="row").sort("row")
    
    df_products = pl.from_records(products, schema=["col", "row", "value"], orient="row")
    df_products = df_products.pivot(values="value", on="col", index="row").sort("row")

    df_products = df_products.filter(df_products['row']>=df['row'][0] ,df_products['row']<=df['row'][-1])
    display(df_products.join(df, on="row", how="left"))



In [ ]:
for i, col in enumerate(df.columns):
        cols = ed.cols[report_type].value
        value = df[col][0]        # value from first row
        if(isinstance(value, str)):
            for val in value.split():
                if'/' in val:
                    num = int(val.replace('/', ''))
                    next_cols = df.columns[i : i + cols + 1]
                    cols_to_select = ['1','2', *next_cols] if report_type in ['Operational'] else ['1', *next_cols]
                    if num < date :
                        all_data['report_last_year'] = df.select(cols_to_select)
                        all_data['report_last_year'] = all_data['report_last_year'].with_row_index("row")
                    elif num == date :
                        all_data['report_this_year'] = df.select(cols_to_select)
                        all_data['report_this_year'] = all_data['report_this_year'].with_row_index("row")
                    else:
                        if(val[:4] == str(date)[:4]):
                            all_data['est_remain'] = df.select(cols_to_select)
                            all_data['est_remain'] = all_data['est_remain'].with_row_index("row")
                        else:
                            all_data['est_next_year'] = df.select(cols_to_select)
                            all_data['est_next_year'] = all_data['est_next_year'].with_row_index("row")

In [57]:
#Raw Materials
all_data = create_dict_dataframes(url, date, 'RawMaterial')
data =all_data['report_this_year']
# generate proper column names
data = data.rename({col: f"{i}" for i, col in enumerate(data.columns)})
#data = data.slice(1)

all_data = pl.DataFrame()

for i in range (2,6):  #ستون‌های 1 تا 4 
    pair = str((i+1)*2)  #2->6, 3->8, 4->10, 5->12
    cols = [str(i), pair]
    all_data = pl.concat([all_data, data.select(cols)], how='horizontal')
all_data = all_data.insert_column(0,data['1'])
all_data = all_data.insert_column(0, data['0'])

all_data = all_data.rename({"0": "row"})

translations = {
    "موجودي اول دوره": "Beginning Inventory",
    "موجودی اول دوره": "Beginning Inventory",
    "خريد طي دوره": "Purchases During the Period",
    "خرید طی دوره": "Purchases During the Period",
    "مصرف": "Consumption",
    "موجودي پايان دوره": "Ending Inventory",
    "موجودی پایان دوره": "Ending Inventory",
}

# فقط ستون‌هایی که string هستند
str_cols = [c for c, dt in zip(all_data.columns, all_data.dtypes) if dt == pl.Utf8]

all_data = all_data.with_columns(
    [
        pl.col(c).replace(translations, default=pl.col(c)).alias(c)
        for c in str_cols
    ]
)
header_idx = 1      # ردیفی که عنوان‌ها در آن است (موجودی اول دوره ...)
data_start = 4      # داده واقعی

cols = all_data.columns

# 1) استخراج عنوان ستون ها از ردیف هدر
header_vals = all_data.row(header_idx)
col_to_label = {
    c: v for c, v in zip(cols, header_vals)
    if c not in ["row", "1"] and v not in (None, "")
}

# 2) Pair detection (qty/value)
qty_cols   = []
value_cols = []

for i in range(2, 6):
    qty_cols.append(str(i))            # 2,3,4,5
    value_cols.append(str((i+1)*2))    # 6,8,10,12

# 3) آماده‌سازی دیتا
df = all_data.slice(data_start)

frames = []
for qc, vc in zip(qty_cols, value_cols):
    label = col_to_label.get(qc) or col_to_label.get(vc)

    frames.append(
        df.select(
            "row",
            "1",
            pl.lit(label).alias("label"),
            pl.col(qc).alias("qty"),
            pl.col(vc).alias("value"),
        )
    )

result = pl.concat(frames).sort(["row", "label"])

idx_internal_raw = result.filter(result['1'] == "مواد اولیه داخلی:")['row'][0]
idx_total_internal_raw= result.filter(result['1']== "جمع مواد اولیه داخلی")['row'][0]
df_internal_raw = result.filter(result['row']>idx_internal_raw,result['row']<idx_total_internal_raw-1)
df_internal_raw=df_internal_raw.with_columns(pl.lit("داخلی").alias("type"))
idx_export_raw = result.filter(result['1'] == "مواد اولیه وارداتی:")['row'][0]
idx_total_export_raw= result.filter(result['1']== "جمع مواد اولیه وارداتی")['row'][0]
df_export_raw = result.filter(result['row']>idx_export_raw,result['row']<idx_total_export_raw-1)
df_export_raw=df_export_raw.with_columns(pl.lit("وارداتی").alias("type"))
df_goods = pl.concat([df_internal_raw,df_export_raw])
df_goods



C:\Users\Elmir\AppData\Local\Temp\ipykernel_6972\2537264157.py:34: DeprecationWarning: the `default` parameter for `replace` is deprecated. Use `replace_strict` instead to set a default while replacing values.
(Deprecated in version 1.0.0)
  pl.col(c).replace(translations, default=pl.col(c)).alias(c)


row,1,label,qty,value,type
u32,str,str,str,str,str
5,"""سنگ آهن""","""Beginning Inventory""","""1537623""","""43718175""","""داخلی"""
5,"""سنگ آهن""","""Consumption""","""4027674""","""47287852""","""داخلی"""
5,"""سنگ آهن""","""Ending Inventory""","""844608""","""58235052""","""داخلی"""
5,"""سنگ آهن""","""Purchases During the Period""","""3334659""","""51706568""","""داخلی"""
6,"""آهن قراضه و چدن""","""Beginning Inventory""","""14171""","""142228424""","""داخلی"""
6,"""آهن قراضه و چدن""","""Consumption""","""39306""","""168802804""","""داخلی"""
6,"""آهن قراضه و چدن""","""Ending Inventory""","""4811""","""163578674""","""داخلی"""
6,"""آهن قراضه و چدن""","""Purchases During the Period""","""29946""","""180539004""","""داخلی"""
7,""" اسفنجی خریداری شده""","""Beginning Inventory""","""56992""","""132574589""","""داخلی"""


In [24]:
col_names = ['Symbol','Date','Period', 'Publish','goods','type','lable','QTY','value']

In [ ]:
all_data = create_dataframe(symbol, cells_tuples, reports[date], date)
report_df   = pl.DataFrame()
estimate_df = pl.DataFrame()
for key,value in all_data.items():
    if 'est' in key:
        estimate_df = pl.concat([estimate_df, convert_dataframe_datatype(value)])
    else:
        report_df   = pl.concat([report_df, convert_dataframe_datatype(value)])

### Create KEY -> date+symbol_code+product+type

In [ ]:
conn=sqlite3.connect("codal.sqlite")
cur = conn.cursor()
rows = conn.execute("SELECT name, id FROM Product").fetchall()
lookup = {pid: product for pid, product in rows}
all_data['report_this_year'].with_columns(
    pl.col("Product").replace(lookup).alias("Product_id"))


insert into DB

In [ ]:
conn=sqlite3.connect("codal.sqlite")
cur = conn.cursor()
cur.execute('''drop table if exists Symbol''')
cur.execute('''create table Symbol(id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT)''')
cur.execute("INSERT INTO Symbol (id, name) VALUES (?, ?)", (1010,"فخوز"))
conn.commit()
conn.close()

In [ ]:
units=["کیلوگرم","تن"]
ids =[1,2]
conn=sqlite3.connect("codal.sqlite")
cur = conn.cursor()
cur.execute('''drop table if exists Unit''')
cur.execute('''create table Unit(id INTEGER,
            name TEXT)''')
for prod, id in zip(units, ids):
    cur.execute("INSERT INTO Unit (id, name) VALUES (?, ?)", (id, prod))
conn.commit()
conn.close()

In [ ]:
products = ["اسلب", "بلوم ، بیلت ", "آهن اسفنجی", "گندله "]
ids = [1001, 1002, 1003, 1004]
conn=sqlite3.connect("codal.sqlite")
cur = conn.cursor()
cur.execute('''drop table if exists Product''')
cur.execute('''create table Product(id INTEGER,
            name TEXT)''')
for prod, id in zip(products, ids):
    cur.execute("INSERT INTO Product (id, name) VALUES (?, ?)", (id, prod))
conn.commit()
conn.close()

In [ ]:
types = ["داخلی", "صادراتی"]
ids=[1,2]
conn=sqlite3.connect("codal.sqlite")
cur=conn.cursor()
cur.execute(""" drop table if exists Type""")
cur.execute("""create table Type(id INTEGER, name TEXT)""")
for typ, id in zip(types,ids):
    cur.execute("INSERT INTO type (id,name)values(?,?)",(id,typ))
conn.commit()
conn.close()


In [ ]:
def insert_df_to_db(df: pl.DataFrame, table_name: str) -> str:
    conn=sqlite3.connect("codal.sqlite")
    cur = conn.cursor()
    cols = df.columns
    rows = df.to_numpy().tolist()
    placeholders = ",".join("?" for _ in cols)
    sql = f'INSERT INTO "{table_name}" ({", ".join(cols)}) VALUES ({placeholders})'
    cur.executemany(sql, rows)
    conn.commit()
    conn.close()
    print(f"{table_name}: inserted {len(rows)} rows")


In [ ]:
cols = ['key','symbol','date','period','publish','product','type','unit','production','sales', 'price']
report_df.columns = cols

In [ ]:
estimate_df

In [ ]:
try:
    insert_df_to_db(report_df, 'report_year')
except Exception as e:
    print(e)
    #continue

database is locked


In [ ]:
conn=sqlite3.connect("codal.sqlite")
cur=conn.cursor()
cur.execute('''drop table if exists report_year''')
cur.execute('''create table report_year(key INTEGER,
            symbol INTEGER, date INTEGER, period INTEGER,
            publish INTEGER, product INTEGER,type INTEGER,
            unit INTEGER, production INTEGER, sales INTEGER, price INTEGER)''')
cur.execute('''drop table if exists EST_year''')
cur.execute('''create table EST_year(key INTEGER,
            symbol INTEGER, date INTEGER, period INTEGER,
            publish INTEGER, product INTEGER,type INTEGER,
            unit INTEGER, production INTEGER, sales INTEGER, price INTEGER)''')

conn.commit()
conn.close()



In [ ]:
#cols = ["key", "symbol", "date", "period", "publish","product", "unit", "type", "production", "Sales"]

for table_name, df in all_data.items():
    if df.height == 0:
        continue

    df_insert = df.select(cols)
    rows = df_insert.rows()
    placeholders = ",".join("?" for _ in cols)

    cur.executemany(
        f"INSERT INTO {table_name} ({', '.join(cols)}) VALUES ({placeholders})",
        rows
    )